In [ ]:
import serial
import time

port = "COM3" 
baudrate = 9600 

ser = serial.Serial(
    port=port,
    baudrate=baudrate,
    bytesize=serial.EIGHTBITS,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    timeout=1
)

instructions = [
    "00000000000000000000010100010011",
    "00000000000100000000010110010011",
    "00000000101000000000011000010011",
    "00000000000001100000110001100011",
    "00000000101101010000001010110011",
    "00000000101100000000010100110011",
    "00000000010100000000010110110011",
    "11111111111101100000011000010011",
    "11111110110111111111000001101111",
    "00000000000000000000000001101111"
]

time.sleep(0.1)

ser.write(bytes(10))
for inst in instructions:
    b0 = int(inst[0:8],   2)
    b1 = int(inst[8:16],  2)
    b2 = int(inst[16:24], 2)
    b3 = int(inst[24:32], 2)
    ser.write(bytes([b0, b1, b2, b3]))

ser.flush()

print("Listo. Datos enviados.")
ser.close()

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (2683830391.py, line 6)